In [2]:
import os
import re
import pandas as pd

In [11]:
import os
import re
import pandas as pd

folder_path = '../Simulation/Assets/Results/Correlation_cost_completiontime_modidied_A_ver'

column_name = ['File_name', 'Type',  'C_max', 'C_avg']
values = []

# get folder
for folder_name in os.listdir(folder_path):
    if not folder_name.endswith('meta'):
        cost_type = folder_name.split('_')[-1]
        if not cost_type[0].isupper():
            cost_type = cost_type[0].upper() + cost_type[1:]

        # get files in folder
        folder_full_path = os.path.join(folder_path, folder_name)

        # Iterate over files in the folder
        for file_name in os.listdir(folder_full_path):
            if not file_name.endswith('meta'):
                # 정규 표현식을 사용하여 'prev'와 'now'를 기준으로 파일 이름 분리
                match = re.match(r'result_(prev.*?)_with_(now.*?)\.csv', file_name)
                if match:
                    prev_part  = match.group(1)
                    now_part = match.group(2) + '.csv'

                    # "RoutePoints"를 "Truck"으로 바꿈
                    prev_file_name = prev_part.replace("RoutePoints", "Truck")
                    now_file_name = now_part.replace("RoutePoints", "Truck")

                    # read csv file
                    file_full_path = os.path.join(folder_full_path, file_name)
                    df = pd.read_csv(file_full_path)
                    # print(df.columns)
                    c_max_prev = df['C_max_prev'][0]
                    c_max_now = df['C_max_now'][0]

                    c_avg_prev = df.loc[df['Truck_id'].str[6:].astype(int)<100, 'Completion_Time'].mean()
                    c_avg_now = df.loc[df['Truck_id'].str[6:].astype(int)>=100, 'Completion_Time'].mean()

                    # Append values to the list
                    prev_value = [prev_file_name, cost_type, c_max_prev, c_avg_prev]
                    values.append(prev_value)
                    now_value = [now_file_name, cost_type, c_max_now, c_avg_now]
                    values.append(now_value)

# create dataframe
cost_type_df = pd.DataFrame(values, columns=column_name)
print(cost_type_df)


                              File_name Type    C_max     C_avg
0    prev_Truck_20_LP_0_0_100_10rep.csv  Max  487.432  433.1844
1     now_Truck_20_LP_0_0_100_10rep.csv  Max  512.512  448.3916
2    prev_Truck_20_LP_0_0_100_11rep.csv  Max  482.400  436.1644
3     now_Truck_20_LP_0_0_100_11rep.csv  Max  465.632  420.1980
4    prev_Truck_20_LP_0_0_100_12rep.csv  Max  497.048  434.8400
..                                  ...  ...      ...       ...
235   now_Truck_20_LP_40_50_10_7rep.csv  Min  431.848  385.6760
236  prev_Truck_20_LP_40_50_10_8rep.csv  Min  419.640  385.6288
237   now_Truck_20_LP_40_50_10_8rep.csv  Min  466.832  386.6352
238  prev_Truck_20_LP_40_50_10_9rep.csv  Min  398.320  374.5036
239   now_Truck_20_LP_40_50_10_9rep.csv  Min  406.800  374.6804

[240 rows x 4 columns]


In [12]:
cost_type_df[cost_type_df['File_name'] == 'prev_Truck_20_LP_0_0_100_1rep.csv']

,File_name,Type,C_max,C_avg
20,prev_Truck_20_LP_0_0_100_1rep.csv,Max,479.992,425.8480
100,prev_Truck_20_LP_0_0_100_1rep.csv,Middle,456.280,406.4500
180,prev_Truck_20_LP_0_0_100_1rep.csv,Min,407.896,377.6852


In [13]:
original_file_name = 'C:\\Users\\USER\\workspace\\TSB\\Model\\corrlation_data_modified_ver_edited.csv'

original_df = pd.read_csv(original_file_name)


# type열 공백제거
original_df['Type'] = original_df['Type'].str.strip()
print(original_df)

                               File_name    Type  Objective_value
0      prev_Truck_20_LP_0_0_100_1rep.csv     Min            63800
1       now_Truck_20_LP_0_0_100_1rep.csv     Min            68000
2     prev_Truck_20_LP_40_50_10_1rep.csv     Min            34751
3      now_Truck_20_LP_40_50_10_1rep.csv     Min            34924
4      prev_Truck_20_LP_0_0_100_1rep.csv  Middle            98400
..                                   ...     ...              ...
235   now_Truck_20_LP_40_50_10_20rep.csv  Middle            48319
236   prev_Truck_20_LP_0_0_100_20rep.csv     Max           112800
237    now_Truck_20_LP_0_0_100_20rep.csv     Max           115600
238  prev_Truck_20_LP_40_50_10_20rep.csv     Max            59834
239   now_Truck_20_LP_40_50_10_20rep.csv     Max            62918

[240 rows x 3 columns]


In [14]:
original_df[original_df['File_name'] == 'prev_Truck_20_LP_0_0_100_1rep.csv']

,File_name,Type,Objective_value
0,prev_Truck_20_LP_0_0_100_1rep.csv,Min,63800
4,prev_Truck_20_LP_0_0_100_1rep.csv,Middle,98400
8,prev_Truck_20_LP_0_0_100_1rep.csv,Max,119600


In [15]:
# 'File_name'과 'Type'이 같은 행끼리 병합
merged_df = pd.merge(original_df, cost_type_df, on=['File_name', 'Type'], how='inner')
print(merged_df)

                               File_name    Type  Objective_value    C_max  \
0      prev_Truck_20_LP_0_0_100_1rep.csv     Min            63800  407.896   
1       now_Truck_20_LP_0_0_100_1rep.csv     Min            68000  424.416   
2     prev_Truck_20_LP_40_50_10_1rep.csv     Min            34751  404.464   
3      now_Truck_20_LP_40_50_10_1rep.csv     Min            34924  426.384   
4      prev_Truck_20_LP_0_0_100_1rep.csv  Middle            98400  456.280   
..                                   ...     ...              ...      ...   
235   now_Truck_20_LP_40_50_10_20rep.csv  Middle            48319  465.496   
236   prev_Truck_20_LP_0_0_100_20rep.csv     Max           112800  501.624   
237    now_Truck_20_LP_0_0_100_20rep.csv     Max           115600  515.896   
238  prev_Truck_20_LP_40_50_10_20rep.csv     Max            59834  500.584   
239   now_Truck_20_LP_40_50_10_20rep.csv     Max            62918  534.880   

        C_avg  
0    377.6852  
1    373.7180  
2    372.8872  

In [16]:
merged_df[merged_df['File_name'] == 'prev_Truck_20_LP_0_0_100_1rep.csv']

,File_name,Type,Objective_value,C_max,C_avg
0,prev_Truck_20_LP_0_0_100_1rep.csv,Min,63800,407.896,377.6852
4,prev_Truck_20_LP_0_0_100_1rep.csv,Middle,98400,456.280,406.4500
8,prev_Truck_20_LP_0_0_100_1rep.csv,Max,119600,479.992,425.8480


In [17]:
# dataframe to csv
merged_df.to_csv('correlation_data_with_C_max_modified_ver.csv', index=False)